In [1]:
import pandas as pd
import numpy as np

In [2]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
from bs4 import BeautifulSoup as bs
import requests

In [4]:
import datetime
import time
from datetime import date

In [5]:
import os, sys

In [6]:
# !pip install chromedriver_autoinstaller

import chromedriver_autoinstaller

driver_name = chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path
time.sleep(1)

In [7]:
driver = wd.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - Downloading:  95%|█████████▍| 5.96M/6.30M [00:00<00:00, 11.9MB/s]

In [8]:
def pick_date_in_calendar(start_date, end_date):
    driver.find_element(By.CLASS_NAME, 'DatePickerWrapper_icon__Qw9f8').click()

    start = driver.find_elements(By.CLASS_NAME, 'NativeDateInput_root__wbgyP')[0]
    end = driver.find_elements(By.CLASS_NAME, 'NativeDateInput_root__wbgyP')[1]

    start_element = start.find_element(By.CSS_SELECTOR, 'input[type=date]')
    end_element = end.find_element(By.CSS_SELECTOR, 'input[type=date]')

    start_element.clear()
    start_element.send_keys(start_date)
    start_element.click()

    end_element.clear()
    end_element.send_keys(end_date)
    end_element.click()

    driver.find_element(By.CLASS_NAME, 'inv-button.HistoryDatePicker_apply-button__fPr_G').click()

In [9]:
main_url = 'https://www.investing.com/indices/us-30-historical-data'
driver.get(main_url)
time.sleep(1)

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:20<00:00, 11.9MB/s]

In [12]:
today = datetime.date.today()
today_str = today.strftime('%Y-%m-%d')

In [13]:
today_str

'2023-06-14'

In [15]:
pick_date_in_calendar(start_date, today_str)

In [14]:
start_date = '2021-12-20'

In [16]:
table_class = 'datatable_table__D_jso datatable_table--border__B_zW0 datatable_table--mobile-basic__W2ilt datatable_table--freeze-column__7YoIE'
df = pd.read_html(driver.page_source, attrs={"class": table_class}, flavor=["lxml", "bs4"])[0]

In [18]:
df.tail()

,Date,Price,Open,High,Low,Vol.,Change %
367,12/27/2021,36302.38,35954.48,36306.61,35954.48,250.26M,+0.98%
368,12/23/2021,35950.56,35782.42,36060.99,35782.42,250.94M,+0.55%
369,12/22/2021,35753.89,35491.71,35764.67,35433.13,290.73M,+0.74%
370,12/21/2021,35492.70,35069.50,35508.96,35069.50,349.67M,+1.60%
371,12/20/2021,34932.16,35222.12,35222.12,34665.50,388.89M,-1.23%


In [33]:
rename_dict = {'Date': 'date', 'Price': 'us-30', 'Open':'open', 'High': 'high', 'Low':'low', 'Vol.':'vol', 'Change %':'cr'}

In [19]:
df['Date'] = df['Date'].apply(lambda x : datetime.datetime.strptime(x, "%m/%d/%Y"))
df = df.sort_values(by='Date').reset_index(drop=True) # ascending order and renumbering index starting 0.

In [32]:
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2021-12-20,34932.16,35222.12,35222.12,34665.50,388.89M,-1.23%
1,2021-12-21,35492.70,35069.50,35508.96,35069.50,349.67M,+1.60%
2,2021-12-22,35753.89,35491.71,35764.67,35433.13,290.73M,+0.74%
3,2021-12-23,35950.56,35782.42,36060.99,35782.42,250.94M,+0.55%
4,2021-12-27,36302.38,35954.48,36306.61,35954.48,250.26M,+0.98%


In [36]:
df['Vol.'] = df['Vol.'].str[:-1].astype('float')

In [38]:
df['Change %'] = df['Change %'].str[:-1].astype('float')

In [39]:
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2021-12-20,34932.16,35222.12,35222.12,34665.50,388.89,-1.23
1,2021-12-21,35492.70,35069.50,35508.96,35069.50,349.67,1.60
2,2021-12-22,35753.89,35491.71,35764.67,35433.13,290.73,0.74
3,2021-12-23,35950.56,35782.42,36060.99,35782.42,250.94,0.55
4,2021-12-27,36302.38,35954.48,36306.61,35954.48,250.26,0.98


In [40]:
driver.close()
driver.quit()